In [28]:
import requests
import json
from tqdm import tqdm
import os
import git
import shutil
import stat


In [8]:
# https://github.com/search?l=&p=67&q=language%3ASolidity&type=Repositories
url = f"https://api.github.com/search/repositories?q=language:Solidity&per_page=100&page={}"
response = requests.get(url)
with open('github_solidity_repo.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(response.json(), indent=2))

In [23]:
for repo in tqdm(response.json()['items']):
    clone_url = repo['clone_url']
    repo_name = repo['name']
    path = os.path.join(os. getcwd(), 'data', repo_name)
    try:
        os.mkdir(path)
    except FileExistsError:
        continue
    try:
        git.Repo.clone_from(clone_url, path)
    except git.exc.GitCommandError:
        print(f'{repo_name} has a GitCommandError. Skipped.')
        continue

 52%|█████▏    | 52/100 [01:42<03:07,  3.90s/it]

Journey-to-become-a-Blockchain-Engineer has a GitCommandError. Skipped.


 77%|███████▋  | 77/100 [02:12<00:26,  1.15s/it]

book has a GitCommandError. Skipped.


100%|██████████| 100/100 [02:37<00:00,  1.57s/it]


In [29]:
# deal with read-only .git folder
def on_rm_error(func, path, exc_info):
    os.chmod(path, stat.S_IWRITE)
    os.unlink(path)

path = os.path.join(os.getcwd(), 'data')
for root, dirs, files in os.walk(path):
    # remove .git folder
    for d in dirs:
        if d == '.git':
            shutil.rmtree(os.path.join(root, d), onerror=on_rm_error)
    # remove files not solidity
    for f in files:
        if not f.lower().endswith('.sol'):
            os.remove(os.path.join(root, f))